# Chunking Strategies II: Semantic chunking

In [8]:
%load_ext autoreload
%autoreload 2

[autoreload of _virtualenv failed: Traceback (most recent call last):
  File "c:\Users\manua\Documents\repos\dslabs\dslab-docai\.venv\lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\manua\Documents\repos\dslabs\dslab-docai\.venv\lib\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
  File "C:\Users\manua\AppData\Roaming\uv\python\cpython-3.10.18-windows-x86_64-none\lib\importlib\__init__.py", line 166, in reload
    spec = module.__spec__ = _bootstrap._find_spec(name, pkgpath, target)
  File "<frozen importlib._bootstrap>", line 945, in _find_spec
  File "c:\Users\manua\Documents\repos\dslabs\dslab-docai\.venv\lib\site-packages\_virtualenv.py", line 51, in find_spec
    if fullname in _DISTUTILS_PATCH and self.fullname is None:
NameError: name '_DISTUTILS_PATCH' is not defined
]
[autoreload of typing_extensions failed: Traceback (most recent call las

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import os
from pathlib import Path
from dotenv import load_dotenv
import tiktoken
from langchain_core.documents import Document
from IPython.display import display, Markdown

from src import utils

# Params

In [10]:
TOKENIZER_NAME ="cl100k_base"
EMB_MODEL="text-embedding-3-small"

In [11]:
path_data = Path() / "data"
path_input = path_data / "interim"
path_output = path_data / "processed" / "chunks"  # Store chunks by experiment

path_output.mkdir(exist_ok=True, parents=True)

# Environment Variables

In [12]:
load_dotenv()

True

In [13]:
encoding = tiktoken.get_encoding(TOKENIZER_NAME)

# Load Data

In [14]:
import json
from langchain_core.documents import Document
dc_chunks = utils.load_documents(path_output / "pages" / "Althera.json")

corpus_doc = Document(
    page_content="\n\n".join([doc.page_content for doc in dc_chunks]),
    metadata={"source": "Althera"}
)

# Semantic Chunking (Experimental)

[5 Levels Of Text Splitting. gkamradt](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb)

In [17]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model=EMB_MODEL,
)
text_splitter_sema90 = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile",  # default
    breakpoint_threshold_amount =0.9,  # default:  0.95
    min_chunk_size=128  # default: None
    )

text_splitter_sema98 = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile", 
    breakpoint_threshold_amount =0.98,  
    min_chunk_size=128  
    )

In [26]:
corpus_sema90  = text_splitter_sema90.split_documents([corpus_doc])
stats_sema90 = utils.corpus_stats(encoding, corpus_sema90)
stats_sema90.describe()

count    138.000000
mean      65.152174
std       42.336832
min        3.000000
25%       47.000000
50%       58.500000
75%       72.000000
max      396.000000
dtype: float64

In [ ]:
# corpus_sema98  = text_splitter_sema98.split_documents([corpus_doc])
# stats_sema98 = utils.corpus_stats(encoding, corpus_sema98)
# stats_sema98.describe()
# count    138.000000
# mean      65.152174
# std       42.336832
# min        3.000000
# 25%       47.000000
# 50%       58.500000
# 75%       72.000000
# max      396.000000
# dtype: float64

In [22]:
# for i, doc in enumerate(corpus_sema98[:10]):
#     print(f"chunk: {i} ----------------")
#     display(Markdown(doc.page_content))

In [27]:
utils.save_documents(
    corpus_sema90,
    path_output /  "sema" / "Althera.json"
)